In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime
from scipy import stats
from scipy.stats import linregress
from config import weather_api_key
from config import g_key

In [9]:
from citipy import citipy

In [10]:
Houston_lat = 29.756097853207656
Houston_lng = -95.3669907972107



lat = np.random.uniform(Houston_lat-1, Houston_lat+2, size=6500)
lng = np.random.uniform(Houston_lng-5, Houston_lng+2, size=6500)
city = []
for x in range(len(lat)):
    city.append(citipy.nearest_city(lat[x], lng[x]).city_name)


In [11]:
df = pd.DataFrame({
    "City":city,
    "lat":lat,
    "lng":lng,
})

In [12]:
df

,City,lat,lng
0,kerrville,30.263267,-99.439187
1,beeville,28.794976,-97.602015
2,huntsville,31.087838,-95.400328
3,copperas cove,30.967913,-97.940588
4,natchitoches,31.108799,-93.515919
...,...,...,...
6495,bay city,28.946359,-96.166285
6496,college station,30.655136,-96.310558
6497,port arthur,29.578321,-93.514052
6498,galveston,29.644613,-94.488223


In [13]:
len(list(df["City"].unique()))

111

In [21]:
target_cities=list(df["City"].unique())
print(target_cities)

['kerrville', 'beeville', 'huntsville', 'copperas cove', 'natchitoches', 'fredericksburg', 'temple', 'brownwood', 'san angelo', 'uvalde', 'austin', 'lufkin', 'lockhart', 'port arthur', 'sulphur', 'canyon lake', 'taylor', 'lumberton', 'robinson', 'vidor', 'woodway', 'nacogdoches', 'conroe', 'hondo', 'liberty', 'galveston', 'texas city', 'bryan', 'gatesville', 'port lavaca', 'wharton', 'pearland', 'lakeway', 'victoria', 'bellmead', 'groves', 'leon valley', 'bay city', 'corsicana', 'beaumont', 'seguin', 'del rio', 'brenham', 'eagle pass', 'new braunfels', 'palestine', 'el campo', 'jacksonville', 'channelview', 'live oak', 'lake jackson', 'pflugerville', 'houston', 'kyle', 'tomball', 'rosenberg', 'angleton', 'pleasanton', 'aldine', 'stephenville', 'la marque', 'orange', 'college station', 'pecan grove', 'humble', 'leander', 'freeport', 'killeen', 'nederland', 'spring', 'georgetown', 'converse', 'richmond', 'atascocita', 'katy', 'missouri city', 'santa fe', 'hewitt', 'mission bend', 'san an

In [7]:
# !python.exe -m pip install --upgrade pip
# !pip install citipy


In [42]:
#get today and previous 4 days' weather data
def unix_time():
    today = datetime.date.today()
    days = [1,2,3,4]
    times = [int(time.mktime(today.timetuple()))]
    for day in days:
        x=today - datetime.timedelta(days=day)
        unixtime = int(time.mktime(x.timetuple()))
        times.append(unixtime)
    return (times)

In [43]:
unix_time()

[1630126800, 1630040400, 1629954000, 1629867600, 1629781200]

In [45]:
base_url2 = ('https://maps.googleapis.com/maps/api/geocode/json')
lat_weather= []
lng_weather=[]

def find_location(target_cities):
    unix_time()
    for city in target_cities[:3]:
        target_city = city
        # base_url = "https://maps.googleapis.com/maps/api/geocode/json"
        req = {"key": g_key,"address": target_city }
        response = requests.get(base_url2, params=req)
        cities = response.json()
        lat = cities["results"][0]["geometry"]["location"]["lat"]
        lng = cities["results"][0]["geometry"]["location"]["lng"]
    #     lat_lng_weather.append(lat)

        for x in times:
            url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?{lat}&{lng}&dt={x}&appid={weather_api_key}"
    #     url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=60.99&lon=30.9&dt={x}&appid={weather_api_key}"
            print(url)
        print ("--"*30)
        


In [47]:
find_location(target_cities)

http://api.openweathermap.org/data/2.5/onecall/timemachine?30.0474332&-99.1403189&dt=1630126800&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?30.0474332&-99.1403189&dt=1630040400&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?30.0474332&-99.1403189&dt=1629954000&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?30.0474332&-99.1403189&dt=1629867600&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?30.0474332&-99.1403189&dt=1629781200&appid=2619cb1052a25e553478de5c60d2a13f
------------------------------------------------------------
http://api.openweathermap.org/data/2.5/onecall/timemachine?28.4008319&-97.7483312&dt=1630126800&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?28.4008319&-97.7483312&dt=1630040400&appid=2619cb1052a25e553478de5c60d

In [ ]:
for x in times:
    url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?{lat}&{lng}&dt={x}&appid={weather_api_key}"
#     url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=60.99&lon=30.9&dt={x}&appid={weather_api_key}"

    print(url)

In [ ]:
Houston_lat = 29.756097853207656
Houston_lng = -95.3669907972107


    
url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?{lat}&{lng}&dt={st}&appid={weather_api_key}"

In [ ]:
print(url)

In [ ]:
new_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=11.7&lon=-99.4&dt=1629522000&units=imperial&appid={weather_api_key}"

In [ ]:
print(new_url)